In [1]:
import time
import numpy as np
import pandas as pd

## Data Collection

In [2]:
from robotics import Robot, reset, release

reset()

robot = Robot()

Simulation stopped...
Simulation started...
Connected to remoteApi server.
 Pioneer_p3dx_ultrasonicSensor1 connected.
 Pioneer_p3dx_ultrasonicSensor2 connected.
 Pioneer_p3dx_ultrasonicSensor3 connected.
 Pioneer_p3dx_ultrasonicSensor4 connected.
 Pioneer_p3dx_ultrasonicSensor5 connected.
 Pioneer_p3dx_ultrasonicSensor6 connected.
 Pioneer_p3dx_ultrasonicSensor7 connected.
 Pioneer_p3dx_ultrasonicSensor8 connected.
 Pioneer_p3dx_ultrasonicSensor9 connected.
 Pioneer_p3dx_ultrasonicSensor10 connected.
 Pioneer_p3dx_ultrasonicSensor11 connected.
 Pioneer_p3dx_ultrasonicSensor12 connected.
 Pioneer_p3dx_ultrasonicSensor13 connected.
 Pioneer_p3dx_ultrasonicSensor14 connected.
 Pioneer_p3dx_ultrasonicSensor15 connected.
 Pioneer_p3dx_ultrasonicSensor16 connected.
 Vision sensor connected.
 Laser connected.
 Left motor connected.
 Right motor connected.
 Robot connected.


In [3]:
def read_data(robot):
    timestamp = time.time()
    command = getattr(robot, "command", "")
    position = robot.get_current_position()
    orientation = robot.get_current_orientation()
    ultrasonic = robot.read_ultrasonic_sensors()
    laser = robot.read_laser()
    
    return [
        timestamp,    # 1 column
        command,      # 1 column
        *position,    # 3 columns
        *orientation, # 3 columns
        *ultrasonic,  # 16 columns
        *laser,       # 1239 columns (not always)
    ]

data = read_data(robot)

len(data)

1263

In [4]:
def move(robot, distance, angular_velocity=1.0, verbose=True):
    if verbose:
        direction = "forward" if angular_velocity > 0 else "backward"
        print(f"Moving {direction} {distance}m at {abs(angular_velocity)}rad/s")
    robot.command = f"move {distance} {angular_velocity}"
    T = distance / (abs(angular_velocity) * robot.WHEEL_RADIUS)
    robot.set_right_velocity(angular_velocity)
    robot.set_left_velocity(angular_velocity)
    time.sleep(T)
    robot.stop()
    time.sleep(0.5)
    robot.command = ""

def rotate(robot, angle=90, angular_velocity=0.1, verbose=True):
    if verbose:
        direction = "left" if angle * angular_velocity > 0 else "right"
        print(f"Rotating {direction} {abs(angle)}° at {abs(angular_velocity)}rad/s")
    robot.command = f"rotate {angle} {angular_velocity}"
    angle = angle * np.pi / 180 # rads
    distance = abs(angle) * robot.ROBOT_WIDTH / 2
    T = distance / (abs(angular_velocity) * robot.WHEEL_RADIUS)
    if angle < 0:
        angular_velocity = -angular_velocity
    robot.set_right_velocity(angular_velocity)
    robot.set_left_velocity(-angular_velocity)
    time.sleep(T - 0.2) # reduce overshoot
    robot.stop()
    time.sleep(0.5)
    robot.command = ""

robot.stop()
move(robot, 1.0)
rotate(robot, -90)
move(robot, 2.0)
rotate(robot, 90)
move(robot, 4.0)
rotate(robot, 90)
move(robot, 4.0)
rotate(robot, 90)
move(robot, 4.0)
rotate(robot, 90)
move(robot, 2.0)
rotate(robot, 90)

Moving forward 1.0m at 1.0rad/s
Rotating right 90° at 0.1rad/s
Moving forward 2.0m at 1.0rad/s
Rotating left 90° at 0.1rad/s
Moving forward 4.0m at 1.0rad/s
Rotating left 90° at 0.1rad/s
Moving forward 4.0m at 1.0rad/s
Rotating left 90° at 0.1rad/s
Moving forward 4.0m at 1.0rad/s
Rotating left 90° at 0.1rad/s
Moving forward 2.0m at 1.0rad/s
Rotating left 90° at 0.1rad/s


In [5]:
import threading

class DataCollector(threading.Thread):
    def __init__(self, robot, read_interval=0.5):
        threading.Thread.__init__(self)
        self.robot = robot
        self.read_interval = read_interval
        self.terminate = False
        self.data = list()

    def stop(self):
        self.terminate = True
    
    def run(self):
        self.terminate = False
        self.data = list()
        N = int(60 / self.read_interval)
        while not self.terminate:
            n = len(self.data) + 1
            if n % N == 0:
                print(f"Collecting data {n}...")
            data = read_data(self.robot)
            self.data.append(data)
            time.sleep(self.read_interval)

dc = DataCollector(robot)

dc.start()

time.sleep(10)

dc.stop()

len(dc.data)

20

In [6]:
class Explore(threading.Thread):
    def __init__(self, robot):
        threading.Thread.__init__(self)
        self.robot = robot

    def run(self):
        robot = self.robot
        robot.stop()
        print("Start.")
        move(robot, 1.0)
        rotate(robot, -90)
        move(robot, 2.0)
        rotate(robot, 90)
        move(robot, 4.0)
        rotate(robot, 90)
        move(robot, 4.0)
        rotate(robot, 90)
        move(robot, 4.0)
        rotate(robot, 90)
        move(robot, 2.0)
        rotate(robot, 90)
        print("Done.")

release(robot)
reset()

robot = Robot()

x = Explore(robot)
x.start()
x.join()

Simulation stopped...
Simulation started...
Connected to remoteApi server.
 Pioneer_p3dx_ultrasonicSensor1 connected.
 Pioneer_p3dx_ultrasonicSensor2 connected.
 Pioneer_p3dx_ultrasonicSensor3 connected.
 Pioneer_p3dx_ultrasonicSensor4 connected.
 Pioneer_p3dx_ultrasonicSensor5 connected.
 Pioneer_p3dx_ultrasonicSensor6 connected.
 Pioneer_p3dx_ultrasonicSensor7 connected.
 Pioneer_p3dx_ultrasonicSensor8 connected.
 Pioneer_p3dx_ultrasonicSensor9 connected.
 Pioneer_p3dx_ultrasonicSensor10 connected.
 Pioneer_p3dx_ultrasonicSensor11 connected.
 Pioneer_p3dx_ultrasonicSensor12 connected.
 Pioneer_p3dx_ultrasonicSensor13 connected.
 Pioneer_p3dx_ultrasonicSensor14 connected.
 Pioneer_p3dx_ultrasonicSensor15 connected.
 Pioneer_p3dx_ultrasonicSensor16 connected.
 Vision sensor connected.
 Laser connected.
 Left motor connected.
 Right motor connected.
 Robot connected.
Start.
Moving forward 1.0m at 1.0rad/s
Rotating right 90° at 0.1rad/s
Moving forward 2.0m at 1.0rad/s
Rotating left 90° a

In [7]:
release(robot)
reset()

robot = Robot()

dc = DataCollector(robot)
dc.start()

x = Explore(robot)
x.start()

x.join()
time.sleep(2)
dc.stop()

len(dc.data)

Simulation stopped...
Simulation started...
Connected to remoteApi server.
 Pioneer_p3dx_ultrasonicSensor1 connected.
 Pioneer_p3dx_ultrasonicSensor2 connected.
 Pioneer_p3dx_ultrasonicSensor3 connected.
 Pioneer_p3dx_ultrasonicSensor4 connected.
 Pioneer_p3dx_ultrasonicSensor5 connected.
 Pioneer_p3dx_ultrasonicSensor6 connected.
 Pioneer_p3dx_ultrasonicSensor7 connected.
 Pioneer_p3dx_ultrasonicSensor8 connected.
 Pioneer_p3dx_ultrasonicSensor9 connected.
 Pioneer_p3dx_ultrasonicSensor10 connected.
 Pioneer_p3dx_ultrasonicSensor11 connected.
 Pioneer_p3dx_ultrasonicSensor12 connected.
 Pioneer_p3dx_ultrasonicSensor13 connected.
 Pioneer_p3dx_ultrasonicSensor14 connected.
 Pioneer_p3dx_ultrasonicSensor15 connected.
 Pioneer_p3dx_ultrasonicSensor16 connected.
 Vision sensor connected.
 Laser connected.
 Left motor connected.
 Right motor connected.
 Robot connected.
Start.
Moving forward 1.0m at 1.0rad/s
Rotating right 90° at 0.1rad/s
Moving forward 2.0m at 1.0rad/s
Rotating left 90° a

719

In [8]:
llen = None
i = 0
n = 0
for i_, data in enumerate(dc.data):
    llen_ = len(data)
    if llen is None:
        llen = llen_
        n = 1
    elif llen != llen_:
        print(llen, i, n)
        llen = llen_
        i = i_
        n = 1
    else:
        n += 1

1269 0 1
1272 1 1
1281 2 1
1275 3 1
1281 4 2
1290 6 1
1299 7 1
1290 8 1
1296 9 1
1299 10 1
1296 11 1
1302 12 1
1305 13 1
1299 14 1
1290 15 1
1287 16 1
1299 17 2
1284 19 1
1269 20 1
1257 21 1
1239 22 1
1230 23 1
1224 24 1
1215 25 1
1203 26 1
1206 27 1
1197 28 1
1209 29 1
1206 30 1
1209 31 3
1212 34 2
1206 36 2
1197 38 1
1212 39 1
1194 40 1
1200 41 1
1206 42 1
1212 43 1
1203 44 1
1179 45 1
1176 46 1
1158 47 1
1155 48 1
1131 49 1
1125 50 1
1110 51 1
1095 52 1
1071 53 1
1083 54 1
1089 55 1
1083 56 1
1068 57 1
1071 58 1
1083 59 1
1077 60 1
1074 61 1
1077 62 2
1089 64 1
1080 65 1
1089 66 1
1092 67 1
1134 68 1
1149 69 1
1158 70 1
1170 71 1
1194 72 1
1197 73 2
1191 75 1
1194 76 2
1215 78 1
1230 79 1
1233 80 1
1248 81 1
1242 82 1
1269 83 1
1278 84 1
1281 85 1
1317 86 1
1284 87 1
1326 88 1
1323 89 1
1317 90 1
1374 91 1
1356 92 1
1392 93 1
1401 94 1
1404 95 1
1440 96 1
1428 97 1
1464 98 1
1476 99 2
1467 101 1
1479 102 1
1494 103 1
1527 104 1
1545 105 1
1563 106 1
1572 107 1
1575 108 1
1584 109 1


In [9]:
columns = ["timestamp", "command", "x", "y", "z", "alpha", "beta", "gamma"]
columns.extend(f"sonar_{i}" for i in range(1, 17))

N = len(columns)

N

24

In [10]:
len(dc.data)

719

In [11]:
sum(1 for data in dc.data if (len(data)-N) % 3)

0

In [12]:
n_cols = max(len(data) for data in dc.data)
n_cols

1977

In [13]:
(n_cols-N) % 3

0

In [14]:
n_laser = (n_cols-N) // 3

n_laser

651

In [15]:
columns.extend(f"laser_{i}_{d}"
               for i in range(1, n_laser+1)
               for d in ["x", "y", "z"])

len(columns)

1977

In [16]:
data = list(np.pad(data, (0,n_cols - len(data)),
                   constant_values=np.nan)
            for data in dc.data)

data[0].shape

(1977,)

In [17]:
data_cols = list(zip(*data))

len(data_cols)

1977

In [18]:
df = pd.DataFrame(data=data, columns=columns)

df.head()

,timestamp,command,x,y,z,alpha,beta,gamma,sonar_1,sonar_2,...,laser_648_z,laser_649_x,laser_649_y,laser_649_z,laser_650_x,laser_650_y,laser_650_z,laser_651_x,laser_651_y,laser_651_z
0,1574808378.2623322,,-2.0253446102142334,-0.8249882459640503,0.13867822289466858,-7.630261097801849e-05,-0.0023941558320075274,-0.0002616322017274797,5.0,5.0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,1574808383.8921142,move 1.0 1.0,-1.5412189960479736,-0.8250638246536255,0.13868993520736694,-5.240448081167415e-05,-0.0015963769983500242,-0.00013539248902816325,5.0,5.0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,1574808384.3951142,move 1.0 1.0,-1.4924519062042236,-0.8250716328620911,0.1387019157409668,-4.9967129598371685e-05,-0.001382801914587617,-0.00015510527009610087,5.0,5.0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,1574808384.8978179,move 1.0 1.0,-1.4437191486358643,-0.8250735998153687,0.13869591057300568,-5.5748554586898535e-05,-0.0014127587201073766,-0.00017283651686739177,5.0,5.0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,1574808385.40241,move 1.0 1.0,-1.3949968814849854,-0.8250824213027954,0.13869857788085938,-7.295125396922231e-05,-0.0015466086333617568,-0.00017355057934764773,5.0,5.0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [19]:
data_file = "sensor-data.csv"
df.to_csv(data_file, index=False)

In [20]:
import os

print(f"File size: {os.stat(data_file).st_size:,d} bytes")

File size: 21,134,939 bytes
